#### **Import Libraries**

In [1]:
!pip install -q -r requirements.txt

In [2]:
%cd ../../extra
!git clone https://github.com/open-mmlab/mmocr.git

d:\HCMAI-AnniVators\extra


Cloning into 'mmocr'...


In [3]:
%cd mmocr
!pip install -qe .

d:\HCMAI-AnniVators\extra\mmocr


  DEPRECATION: Legacy editable install of mmocr==1.0.1 from file:///D:/HCMAI-AnniVators/extra/mmocr (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457


In [4]:
import os
import cv2
import json
import torch
import string
import numpy as np
from PIL import Image
from tqdm import tqdm
from underthesea import text_normalize
from vietocr.tool.config import Cfg
from vietocr.tool.predictor import Predictor
from mmocr.apis import MMOCRInferencer

d:\Miniconda3\envs\aic-env\lib\site-packages\mmengine\optim\optimizer\zero_optimizer.py:11: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import \


In [5]:
%cd ../../dataset

d:\HCMAI-AnniVators\dataset


#### **Parsing Data Path**

In [6]:
def parse_data_path(feature_dir='./distilled_keyframe'):
    all_feature_paths = dict()
    for feature_part in sorted(os.listdir(feature_dir)):
        all_feature_paths[feature_part] = dict()
    for feature_part in sorted(all_feature_paths.keys()):
        feature_part_path = f'{feature_dir}/{feature_part}'
        feature_paths = sorted(os.listdir(feature_part_path))
        feature_ids = [feature_path.split('.')[0] for feature_path in feature_paths]
        for feature_id, feature_path in zip(feature_ids, feature_paths):
            feature_path_full = f'{feature_part_path}/{feature_path}'
            all_feature_paths[feature_part][feature_id] = feature_path_full
    return all_feature_paths

In [7]:
keyframe_dir='./distilled_keyframe'
all_keyframe_paths = parse_data_path(feature_dir=keyframe_dir)

#### **Function Definition**

In [8]:
def sorted_by_id(keyframe_paths):
    id_path_keyframes = []
    for keyframe_path in keyframe_paths:
        keyframe_filename = keyframe_path.split('/')[-1]
        keyframe_id = int(keyframe_filename.split('.')[0])
        id_path_keyframes.append((keyframe_id, keyframe_path))
    sorted_id_path_keyframes = sorted(id_path_keyframes, key=lambda id_path: id_path[0])
    return [id_path[1] for id_path in sorted_id_path_keyframes]

def text_processing(text):
    processed_text = text.lower()
    processed_text = text_normalize(processed_text)
    processed_text = processed_text.translate(str.maketrans('', '', string.punctuation))
    return processed_text

def load_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Không thể đọc ảnh từ đường dẫn: {image_path}")
    return image

def extract_polygons(result_detect):
    if not result_detect or 'predictions' not in result_detect or not result_detect['predictions']:
        return []
    pred = result_detect['predictions'][0]
    return pred.get('det_polygons', [])

def get_top_left_corner(polygon):
    polygon = np.array(polygon).reshape(-1, 2)
    x_min = np.min(polygon[:, 0])
    y_min = np.min(polygon[:, 1])
    return x_min, y_min

def sort_polygons(polygons):
    polygons_with_top_left = [(polygon, *get_top_left_corner(polygon)) for polygon in polygons]
    polygons_sorted = sorted(polygons_with_top_left, key=lambda x: (x[2], x[1]))
    sorted_polygons = [item[0] for item in polygons_sorted]
    return sorted_polygons

def crop_polygon(image, polygon, y_threshold):
    polygon = np.array(polygon).reshape(-1, 2)
    # Find minimum and maximum x, y coordinates
    x_min, y_min = np.min(polygon, axis=0)
    x_max, y_max = np.max(polygon, axis=0)

    if float((y_max-y_threshold)) / (y_max-y_min) > 0.5:
        return None

    # Ensure the coordinates are within the image boundaries
    x_min, y_min = max(0, int(x_min)), max(0, int(y_min))
    x_max, y_max = min(image.shape[1], int(x_max)), min(image.shape[0], int(y_max))
    if x_min >= x_max or y_min >= y_max:
        return None
    return image[int(y_min):int(y_max), int(x_min):int(x_max)]

def recognize_text(cropped_img):
    if cropped_img is None:
        return ""
    cropped_img_pil = Image.fromarray(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
    text_vietocr = vietocr_predictor.predict(cropped_img_pil)
    return text_vietocr.strip()

def det_and_reg(image_path, y_threshold=650):
    image = load_image(image_path)
    if image is None:
        return ""

    result_detect = mmocr_infer(image_path, return_vis=False)
    polygons = extract_polygons(result_detect)
    sorted_polygons = sort_polygons(polygons)

    text_of_image = ""

    for polygon in sorted_polygons:
        if len(polygon) == 0:
            continue

        cropped_image = crop_polygon(image, polygon, y_threshold)
        if cropped_image is None:
            continue

        text = recognize_text(cropped_image)
        if text:
            text_of_image += " " + text

    return text_processing(text_of_image.strip())

def write_json_file(json_data, file_path):
    with open(file_path, 'w') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)

#### **Inference**

In [9]:
# Initialize the MMOCR inferencer
mmocr_infer = MMOCRInferencer(det='drrg')

# VietOCR configuration
config = Cfg.load_config_from_name('vgg_transformer')
config['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'
config['cnn']['pretrained']=False
vietocr_predictor = Predictor(config)

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmocr/textdet/drrg/drrg_resnet50_fpn-unet_1200e_ctw1500/drrg_resnet50_fpn-unet_1200e_ctw1500_20220827_105233-d5c702dd.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std

08/25 09:10:36 - mmengine - WARNING - Failed to search registry with scope "mmocr" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmocr" is a correct scope, or whether the registry is initialized.


d:\Miniconda3\envs\aic-env\lib\site-packages\mmengine\visualization\visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '
d:\Miniconda3\envs\aic-env\lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Model weight C:\Users\LENOVO~1\AppData\Local\Temp\vgg_transformer.pth exsits. Ignore download!


d:\Miniconda3\envs\aic-env\lib\site-packages\vietocr\tool\predictor.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weights, map_locati

In [12]:
save_dir = './filter/ocr'
os.makedirs(save_dir, exist_ok=True)

for video_part, video_path_dict in all_keyframe_paths.items():
    full_save_dir = os.path.join(save_dir, video_part)
    os.makedirs(full_save_dir, exist_ok=True)
    video_ids = sorted(video_path_dict.keys())
    for video_id in tqdm(video_ids):
        json_records = []
        video_id_path = video_path_dict[video_id]
        keyframe_image_paths = [video_id_path + '/' + keyframe_image_path for keyframe_image_path in os.listdir(video_id_path)]
        sorted_keyframe_image_paths = sorted_by_id(keyframe_image_paths)
        for keyframe_image_path in sorted_keyframe_image_paths:
            frame_ocr_result = det_and_reg(keyframe_image_path)
            record = {
                "L": video_part,
                "V": video_id,
                "ID_FRAME": os.path.basename(keyframe_image_path).split('.')[0],  # Giả định rằng ID_FRAME là tên file ~hông có đuôi
                "TEXT": frame_ocr_result
            }
            json_records.append(record)
        write_json_file(json_records, os.path.join(full_save_dir, f'{video_id}.json'))

  0%|          | 0/30 [00:00<?, ?it/s]

Output()